In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = "1"

# Import necessary libraries
import wandb
from transformers import (
    AutoModelForSeq2SeqLM,
    AutoModelForCausalLM,
    AutoTokenizer,
    DataCollatorForSeq2Seq,
    Trainer,
    TrainingArguments,
    Seq2SeqTrainer,
    Seq2SeqTrainingArguments,
    # DataCollatorWithPadding,
    DataCollatorForLanguageModeling,
)
from datasets import load_dataset
import torch
import yaml
from pprint import pprint
from utils import(
    preprocess_function_seq2seq,
    preprocess_function_causal_lm,
    preprocess_function_causal_lm_sft_training,
    preprocess_function_causal_lm_sft_testing,
    preprocess_logits_for_metrics,
    compute_metrics,
    compute_metrics_causal_lm,
    set_seed,
    print_trainable_params_info,
)

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)


# Set up logging and tracking
wandb.login()

# get training configuration
with open('training_config.yaml') as file:
    config = yaml.safe_load(file)

print('-'*50)
print("Training configuration:")
pprint(config)
print('-'*50)

MODELS_DICT = config['MODELS_DICT']

# Training hyperparameters
num_train_epochs = config['hyperparameters']['num_train_epochs']
lr = config['hyperparameters']['lr']
batch_size = config['hyperparameters']['batch_size']
gradient_accumulation_steps = config['hyperparameters']['gradient_accumulation_steps']
max_grad_norm = config['hyperparameters']['max_grad_norm']
warmup_steps = config['hyperparameters']['warmup_steps']
warmup_ratio = config['hyperparameters']['warmup_ratio']

# Logging and saving
logging_steps = config['hyperparameters']['logging_steps']
save_steps = config['hyperparameters']['save_steps']
eval_steps = config['hyperparameters']['eval_steps']

# Training data path
TRAIN_DATA_PATH = config['DATASET_PATH']

# base model path
BASE_MODEL = config['BASE_MODEL']
MODEL_PATH = MODELS_DICT[BASE_MODEL]['MODEL_PATH']
IS_CAUSAL_LM = MODELS_DICT[BASE_MODEL]['CAUSAL_LM']
IS_SFT_TRAINING = MODELS_DICT[BASE_MODEL]['SFT_TRAINING']
FP16_TRAINING = config['FP16_TRAINING']

# max training samples
MAX_TRAINING_SAMPLES = config['MAX_TRAINING_SAMPLES']

if FP16_TRAINING:
    torch_dtype=torch.bfloat16 # bfloat16 has better precission than float16 thanks to bigger mantissa. Though not available with all GPUs architecture.
else:
    torch_dtype=torch.float32

# set seed
SEED = config['SEED']
set_seed(SEED)

# Load dataset
dataset = load_dataset(TRAIN_DATA_PATH)  # Replace with your dataset path

# truncate training dataset to observe data size impact on performance
print(f'[INFO] Truncating training samples to: {MAX_TRAINING_SAMPLES}...')
dataset['train'] = dataset['train'].select(range(min(len(dataset['train']), MAX_TRAINING_SAMPLES)))
print(f'[INFO] Dataset loaded: {dataset}')
print('-'*50)

# Load tokenizer and model
if IS_CAUSAL_LM:
    model = AutoModelForCausalLM.from_pretrained(
        MODEL_PATH,
        torch_dtype=torch_dtype,
    )
else:
    model = AutoModelForSeq2SeqLM.from_pretrained(
        MODEL_PATH,
        torch_dtype=torch_dtype, 
    )
tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)
tokenizer.padding_side = 'left'

if config['hyperparameters']['USE_LORA']:
    # Apply LoRA
    print(f"[INFO] Training with LoRA")
    from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
    
    # Define LoRA configuration
    lora_config = LoraConfig(
        r=config['hyperparameters']['lora_r'],
        lora_alpha=config['hyperparameters']['lora_alpha'],
        lora_dropout=config['hyperparameters']['lora_dropout'],
        bias="none",
        task_type="CAUSAL_LM" if IS_CAUSAL_LM else "SEQ_2_SEQ_LM",  # Adjust for your task
        target_modules=config['hyperparameters']['target_modules'],  # Specify target modules if required
    )
    
    # Wrap the model with LoRA
    model = get_peft_model(model, lora_config)

    # Log trainable parameters for verification
    print_trainable_params_info(model)

    print('-'*50)

# Set a maximum length for tokenization
tokenizer.model_max_length = config['hyperparameters']['MAX_LEN']
if BASE_MODEL == "gpt2":
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})

print(f'[INFO] Model and Tokenizer loaded: {MODEL_PATH}, version: {BASE_MODEL}, IS_SFT_TRAINING: {IS_SFT_TRAINING}, FP16_TRAINING: {FP16_TRAINING}')
print('-'*50)

# Project name for loggings and savings
project_name = "arabic-summarization-v2"
fp16 = '-FP16' if FP16_TRAINING else ''
sft = '-SFT' if IS_SFT_TRAINING else ''
# LoRA params
lora_training = f'-lora' if config['hyperparameters']['USE_LORA'] else ''
lora_r = f'-r-{config['hyperparameters']['lora_r']}' if config['hyperparameters']['USE_LORA'] else ''
lora_alpha = f'-a-{config['hyperparameters']['lora_alpha']}' if config['hyperparameters']['USE_LORA'] else ''
lora_dropout = f'-d-{config['hyperparameters']['lora_dropout']}' if config['hyperparameters']['USE_LORA'] else ''

run_name = 'meh' #f'{MODEL_PATH.split("/")[-1]}-bs-{batch_size}-lr-{lr}-ep-{num_train_epochs}-wp-{warmup_steps}-gacc-{gradient_accumulation_steps}-gnm-{max_grad_norm}{fp16}{sft}-mx-{config['hyperparameters']['MAX_LEN']}{lora_training}{lora_r}{lora_alpha}-v2'
assert '--' not in run_name, f"[WARN] Detected -- in run_name. This will cause a push_to_hub error! Found run_name={run_name} "
assert len(run_name) < 96, f"[WARN] run_name too long. This will cause a push_to_hub error! Consider squeezing it. Found run_name={run_name}"

# Where to save the model
MODEL_RUN_SAVE_PATH = f"BounharAbdelaziz/{run_name}"

# Save the configuration to a .txt file
output_filename = f"./run_configs/{run_name}.txt"
with open(output_filename, 'w') as output_file:
    for key, value in config.items():
        output_file.write(f"{key}: {value}\n")

print(f"Configuration saved to {output_filename}")

# Initialize wandb
wandb.init(
    # set the wandb project where this run will be logged, all runs will be under this project
    project=project_name,   
    # Group runs by model size
    group=MODEL_PATH,       
    # Unique run name
    name=run_name,
    # track hyperparameters and run metadata
    config={
        "learning_rate": lr,
        "num_train_epochs": num_train_epochs,
        "batch_size": batch_size,
        "warmup_ratio": warmup_ratio,
        # "warmup_steps": warmup_steps,
        "max_grad_norm": max_grad_norm,
        "gradient_accumulation_steps": gradient_accumulation_steps,
        # "weight_decay": weight_decay,
        "dataset": TRAIN_DATA_PATH,
    }
)

/home/infres/abounhar/.local/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package wordnet to
[nltk_data]     /home/infres/abounhar/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /home/infres/abounhar/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /home/infres/abounhar/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
wandb: Currently logged in as: abdelazizbounhar. Use `wandb login --relogin` to force relogin


--------------------------------------------------
Training configuration:
{'BASE_MODEL': 'Qwen2.5-0.5B-Instruct',
 'DATASET_PATH': 'BounharAbdelaziz/Arabic-Synthetic-Summarization-Dataset-Filtered',
 'FP16_TRAINING': True,
 'MAX_TRAINING_SAMPLES': 1000,
 'METRIC_FOR_BEST_MODEL': 'rougeL',
 'MODELS_DICT': {'Falcon3-1B-Base': {'CAUSAL_LM': True,
                                     'MODEL_PATH': 'tiiuae/Falcon3-1B-Base',
                                     'SFT_TRAINING': False},
                 'Falcon3-1B-Base-SFT': {'CAUSAL_LM': True,
                                         'MODEL_PATH': 'tiiuae/Falcon3-1B-Base',
                                         'SFT_TRAINING': True},
                 'Falcon3-1B-Instruct': {'CAUSAL_LM': True,
                                         'MODEL_PATH': 'tiiuae/Falcon3-1B-Instruct',
                                         'SFT_TRAINING': True},
                 'Falcon3-3B-Instruct': {'CAUSAL_LM': True,
                                         

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


[INFO] Model and Tokenizer loaded: Qwen/Qwen2.5-0.5B-Instruct, version: Qwen2.5-0.5B-Instruct, IS_SFT_TRAINING: True, FP16_TRAINING: True
--------------------------------------------------
Configuration saved to ./run_configs/meh.txt


In [15]:
# custom instruct prompt start
prompt_template = f"Summarize this arabic text:\n{{text}}\n---\nSummary:\n{{summary}}{{eos_token}}"

# template dataset to add prompt to each sample
def template_dataset(sample):
    sample["text"] = prompt_template.format(text=sample["text"],
                                            summary=sample["summary"],
                                            eos_token=tokenizer.eos_token)
    return sample


# apply prompt template per sample
train_dataset = dataset["train"].map(template_dataset)
eval_dataset = dataset["validation"].map(template_dataset)

# tokenize and chunk dataset
lm_train_dataset = train_dataset.map(
    lambda sample: tokenizer(sample["text"]), batched=True, remove_columns=list(train_dataset.features)
)


lm_eval_dataset = eval_dataset.map(
    lambda sample: tokenizer(sample["text"]), batched=True, remove_columns=list(eval_dataset.features)
)

Map: 100%|██████████| 437/437 [00:00<00:00, 1324.18 examples/s]


In [18]:
lm_train_dataset

Dataset({
    features: ['input_ids', 'attention_mask'],
    num_rows: 1000
})

In [19]:
if IS_CAUSAL_LM:
    
    if IS_SFT_TRAINING:

        # Training arguments
        training_args = TrainingArguments(
            output_dir=MODEL_RUN_SAVE_PATH,
            evaluation_strategy="steps",
            learning_rate=lr,
            warmup_ratio=warmup_ratio,
            per_device_train_batch_size=batch_size,
            per_device_eval_batch_size=batch_size,
            num_train_epochs=num_train_epochs,
            save_total_limit=1,
            bf16=config['FP16_TRAINING'],
            fp16_full_eval=config['FP16_TRAINING'],
            logging_steps=logging_steps,
            save_steps=save_steps,
            eval_steps=eval_steps,
            report_to="wandb",
            push_to_hub=False,
            metric_for_best_model=config['METRIC_FOR_BEST_MODEL'],
            gradient_checkpointing=True,
            load_best_model_at_end=True,
            optim=config['hyperparameters']['optimizer'],
            gradient_checkpointing_kwargs={"use_reentrant": False} if config['hyperparameters']['USE_LORA'] else None,  # Avoids gradient issues in backprop when LoRA is set to True. # https://discuss.huggingface.co/t/how-to-combine-lora-and-gradient-checkpointing-in-whisper/50629
        )
        
        # Initialize Trainer
        trainer = Trainer(
            model=model,
            args=training_args,
            train_dataset=lm_train_dataset,
            eval_dataset=lm_eval_dataset,
            data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False),
            compute_metrics=lambda x : compute_metrics_causal_lm(x, tokenizer),
            preprocess_logits_for_metrics=preprocess_logits_for_metrics, # avoids OOM in eval
        )
        
    else:
        
        print(f'[INFO] Running preprocess_function_causal_lm')
        # Apply preprocessing
        tokenized_dataset_train = dataset['train'].map(lambda x: preprocess_function_causal_lm(x, tokenizer), batched=True)
        tokenized_dataset_validation = dataset['validation'].map(lambda x: preprocess_function_causal_lm(x, tokenizer), batched=True)
        tokenized_dataset_test = dataset['test'].map(lambda x: preprocess_function_causal_lm(x, tokenizer), batched=True)
        
        # Training arguments
        training_args = TrainingArguments(
            output_dir=MODEL_RUN_SAVE_PATH,
            evaluation_strategy="steps",
            learning_rate=lr,
            warmup_ratio=warmup_ratio,
            per_device_train_batch_size=batch_size,
            per_device_eval_batch_size=batch_size,
            num_train_epochs=num_train_epochs,
            save_total_limit=1,
            bf16=config['FP16_TRAINING'],
            fp16_full_eval=config['FP16_TRAINING'],
            logging_steps=logging_steps,
            save_steps=save_steps,
            eval_steps=eval_steps,
            report_to="wandb",
            push_to_hub=False,
            metric_for_best_model=config['METRIC_FOR_BEST_MODEL'],
            gradient_checkpointing=True,
            load_best_model_at_end=True,
            optim=config['hyperparameters']['optimizer'],
            gradient_checkpointing_kwargs={"use_reentrant": False} if config['hyperparameters']['USE_LORA'] else None,  # Avoids gradient issues in backprop when LoRA is set to True. # https://discuss.huggingface.co/t/how-to-combine-lora-and-gradient-checkpointing-in-whisper/50629
        )
    
        # Initialize Trainer
        trainer = Trainer(
            model=model,
            args=training_args,
            train_dataset=tokenized_dataset_train,
            eval_dataset=tokenized_dataset_validation,
            data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False),
            compute_metrics=lambda x : compute_metrics_causal_lm(x, tokenizer),
            preprocess_logits_for_metrics=preprocess_logits_for_metrics, # avoids OOM in eval
        )
    
else:
    
    # Apply preprocessing
    tokenized_dataset_train = dataset['train'].map(lambda x: preprocess_function_seq2seq(x, tokenizer), batched=True)
    tokenized_dataset_validation = dataset['validation'].map(lambda x: preprocess_function_seq2seq(x, tokenizer), batched=True)
    tokenized_dataset_test = dataset['test'].map(lambda x: preprocess_function_seq2seq(x, tokenizer), batched=True)

    # Training arguments
    training_args = Seq2SeqTrainingArguments(
        output_dir=MODEL_RUN_SAVE_PATH,
        evaluation_strategy="steps",
        learning_rate=lr,
        warmup_ratio=warmup_ratio,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        num_train_epochs=num_train_epochs,
        save_total_limit=1,
        predict_with_generate=True,
        logging_steps=logging_steps,
        save_steps=save_steps,
        eval_steps=eval_steps,
        report_to="wandb",
        push_to_hub=False,
        metric_for_best_model=config['METRIC_FOR_BEST_MODEL'],
        gradient_checkpointing=True,
        load_best_model_at_end=True,
    )

    trainer = Seq2SeqTrainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_dataset_train,
        eval_dataset=tokenized_dataset_validation,
        tokenizer=tokenizer,
        data_collator=DataCollatorForSeq2Seq(tokenizer, model=model),
        compute_metrics=lambda x : compute_metrics(x, tokenizer),
    )

/home/infres/abounhar/.local/lib/python3.12/site-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
dataset

In [20]:
# Train the model
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...


Step,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Bleu,Meteor,Bertscore Precision,Bertscore Recall,Bertscore F1
100,2.582500,2.602884,55.083788,38.448823,54.245476,54.241797,3.810823,18.660133,67.762839,70.961604,69.278592
200,2.529300,2.587335,55.606804,38.910257,54.759650,54.720753,3.871437,18.798846,67.902154,71.064663,69.402197
300,2.582000,2.582069,55.405143,38.879908,54.549638,54.533909,3.940570,18.912171,67.831755,71.012572,69.341729
400,2.547800,2.580348,55.257497,38.818235,54.404744,54.371896,3.913501,18.850758,67.641649,70.923031,69.194091
500,2.551100,2.580014,55.278462,38.795217,54.417087,54.376928,3.914574,18.879967,67.650611,70.893994,69.188069


There were missing keys in the checkpoint model loaded: ['lm_head.weight'].


TrainOutput(global_step=500, training_loss=2.569189453125, metrics={'train_runtime': 468.8698, 'train_samples_per_second': 2.133, 'train_steps_per_second': 1.066, 'total_flos': 3942051575514624.0, 'train_loss': 2.569189453125, 'epoch': 1.0})

In [21]:
# Push to Hugging Face Hub
print("[INFO] Preparing to push to hub...")

if config['hyperparameters']['USE_LORA']:
    print("[INFO] Merging LoRA weights before pushing...")
    from peft import merge_and_unload
    model = merge_and_unload(model)

[INFO] Preparing to push to hub...


In [24]:
# Save the model and tokenizer locally before pushing
# trainer.save_model(MODEL_RUN_SAVE_PATH)  # This saves the model, tokenizer, and config
# tokenizer.save_pretrained(MODEL_RUN_SAVE_PATH)

# Push to the hub
print("[INFO] Pushing model and tokenizer to Hugging Face Hub...")
trainer.push_to_hub()
tokenizer.push_to_hub(MODEL_RUN_SAVE_PATH)

[INFO] Pushing model and tokenizer to Hugging Face Hub...


No files have been modified since last commit. Skipping to prevent empty commit.


IsADirectoryError: [Errno 21] Is a directory: 'BounharAbdelaziz/meh'

In [46]:
test_input = """,تشير النصوص إلى أهمية التمارين الرياضية الفترية لتحسين مستوي
"عبد الحكيم حذاقة-الجزائر عاشت الجزائر مرحلة مفصلية في تاريخها خلال الشهور
العشرة الأخيرة من عام 2019، شهدت خلالها ثورة سلمية غير مسبوقة، كانت فيها قوات
الجيش مرافقة لحراك الشعب، دون أن تراق قطرة دم. وأطاحت الثورة بنظام الرئيس عبد
العزيز بوتفليقة بعد عشرين عاما من الحكم، وزج بأقوى رموز عهده من مسؤولين أمنيين
وحكوميين ورجال مال وراء القضبان، حيث شهدت البلاد محاكمات تاريخية بالجملة في حق
هؤلاء. وانتهى المخاض العسير طيلة عشرة أشهر بانتخاب رئيس جديد في 12
ديسمبر/كانون الأول، وسط أجواء مشحونة بين المؤيدين والرافضين لشروط الانتخابات،
حيث لا يزال جزائريون مصرين على الاحتجاج الأسبوعي. وفتحت الوفاة المفاجئة لرئيس
الأركان الفريق أحمد قايد صالح يوم 23 ديسمبر/كانون الأول باب التكهنات حول ملامح
العام الجديد، بالنظر إلى دور الرجل الحاسم في صناعة المرحلة الماضية.
"""
# custom instruct prompt start
prompt_template_test = f"Summarize this arabic text:\n{{text}}\n---\nSummary:\n"

test_prompt = prompt_template_test.format(text=test_input)
print(test_prompt)

Summarize this arabic text:
,تشير النصوص إلى أهمية التمارين الرياضية الفترية لتحسين مستوي
"عبد الحكيم حذاقة-الجزائر عاشت الجزائر مرحلة مفصلية في تاريخها خلال الشهور
العشرة الأخيرة من عام 2019، شهدت خلالها ثورة سلمية غير مسبوقة، كانت فيها قوات
الجيش مرافقة لحراك الشعب، دون أن تراق قطرة دم. وأطاحت الثورة بنظام الرئيس عبد
العزيز بوتفليقة بعد عشرين عاما من الحكم، وزج بأقوى رموز عهده من مسؤولين أمنيين
وحكوميين ورجال مال وراء القضبان، حيث شهدت البلاد محاكمات تاريخية بالجملة في حق
هؤلاء. وانتهى المخاض العسير طيلة عشرة أشهر بانتخاب رئيس جديد في 12
ديسمبر/كانون الأول، وسط أجواء مشحونة بين المؤيدين والرافضين لشروط الانتخابات،
حيث لا يزال جزائريون مصرين على الاحتجاج الأسبوعي. وفتحت الوفاة المفاجئة لرئيس
الأركان الفريق أحمد قايد صالح يوم 23 ديسمبر/كانون الأول باب التكهنات حول ملامح
العام الجديد، بالنظر إلى دور الرجل الحاسم في صناعة المرحلة الماضية.

---
Summary:



In [47]:
# Tokenize and generate
inputs = tokenizer(test_prompt, return_tensors="pt").to("cuda")
outputs = model.generate(
        **inputs,
        # max_new_tokens=256,
        # num_beams=4,
        # early_stopping=True,
        # no_repeat_ngram_size=3,
        # repetition_penalty=1.2,
        eos_token_id=tokenizer.eos_token_id,  # Crucial for stopping
        # pad_token_id=tokenizer.eos_token_id,  # Crucial for stopping
        # do_sample=True,
    )


# Decode the output
summary = tokenizer.decode(outputs[0], skip_special_tokens=True)

# split_text = summary.split("### Assistant:", 1)
# final_pred = split_text[-1].strip() if len(split_text) > 1 else summary
# final_pred = final_pred.replace(tokenizer.eos_token, "")
        
print(f"Test summary: {summary}")
# print(f"final_pred: {final_pred}")

Test summary: Summarize this arabic text:
,تشير النصوص إلى أهمية التمارين الرياضية الفترية لتحسين مستوي
"عبد الحكيم حذاقة-الجزائر عاشت الجزائر مرحلة مفصلية في تاريخها خلال الشهور
العشرة الأخيرة من عام 2019، شهدت خلالها ثورة سلمية غير مسبوقة، كانت فيها قوات
الجيش مرافقة لحراك الشعب، دون أن تراق قطرة دم. وأطاحت الثورة بنظام الرئيس عبد
العزيز بوتفليقة بعد عشرين عاما من الحكم، وزج بأقوى رموز عهده من مسؤولين أمنيين
وحكوميين ورجال مال وراء القضبان، حيث شهدت البلاد محاكمات تاريخية بالجملة في حق
هؤلاء. وانتهى المخاض العسير طيلة عشرة أشهر بانتخاب رئيس جديد في 12
ديسمبر/كانون الأول، وسط أجواء مشحونة بين المؤيدين والرافضين لشروط الانتخابات،
حيث لا يزال جزائريون مصرين على الاحتجاج الأسبوعي. وفتحت الوفاة المفاجئة لرئيس
الأركان الفريق أحمد قايد صالح يوم 23 ديسمبر/كانون الأول باب التكهنات حول ملامح
العام الجديد، بالنظر إلى دور الرجل الحاسم في صناعة المرحلة الماضية.

---
Summary:
تعرضت الجزائر ل períادة محددة خلال السنوات الأخيرة، مع انتشار الث


In [ ]:
# Evaluate on test set
test_results = trainer.evaluate(
    tokenized_dataset_test,
    # compute_metrics=lambda eval_pred: compute_metrics_causal_lm(eval_pred, is_in_test=True)
)
print(f'[INFO] Results on test set: {test_results}')

# # Save the model and tokenizer locally before pushing
# trainer.save_model(MODEL_RUN_SAVE_PATH)  # This saves the model, tokenizer, and config
# tokenizer.save_pretrained(MODEL_RUN_SAVE_PATH)

# # Push to the hub
# print("[INFO] Pushing model and tokenizer to Hugging Face Hub...")
# trainer.push_to_hub()
# tokenizer.push_to_hub(MODEL_RUN_SAVE_PATH)

In [6]:
import pandas as pd
df = pd.read_csv('preds.csv')

In [ ]:
df

In [ ]:
import pandas as pd
df = pd.read_csv('preds.csv')

print(df["Predictions"].iloc[0])

print(df["Labels"].iloc[0])


In [ ]:
print(df["Labels"].iloc[0])